### Открываю tmx-файл в Heartsome TMX Editor. Сначала просто ознакамливаюсь с данными. 

#### Устанавливаю модуль Python для работы с tmx-файлами

In [ ]:
!pip install translate-toolkit

In [1]:
from translate.storage.tmx import tmxfile

In [2]:
#Импортирую tmx файл

with open("testTMX.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'ru', 'eng')

In [3]:
# Функция для проверки юнитов только с цифрами и точками, нужна будет дальше

digitsdot = "1234567890."

def isdotdigit(string):
    for k in string:
        if k in digitsdot:
            for k in digitsdot:
                ;
        else:
            return False
    return True    

char = "1.3.2.3"

print(isdotdigit(char))

True


### Начинаю проверку датасета на пустоты,  на юниты, не имеющие ценности и т.д.

In [5]:
s = 0
t = 0
b = 0
count = 0
dds = 0
ddt = 0
trash = 0
original_count = 0

for node in tmx_file.unit_iter():
    count += 1
    source = node.getid()
    target = node.gettarget()
    
# Проверяем, есть ли в юните пустой source, либо пустой target

    if node.isblank():
        b += 1
        
# Проверяем, является ли source, либо target, исключительно числом    

    if source.isnumeric():
        s += 1
    if target.isnumeric():
        t += 1
        
# Предположим, что все source, в которых меньше, чем 6 символов, не имеют никакой ценности. 
# Посмотрим на такие юниты и посчитаем количество
# Из этих юнитов даже беглым взглядом можно вычленить много мусора: юниты-числа с точками, буквы(не слова) с точками и т.д.
# Их всего 72(1,8% от датасета)

    if len(source) < 6:
        print(source)
        print(target)
        print("________")
        trash += 1

# Проверим, возможно, есть множество юнитов больше 6 символов, где только цифры и точки, т.е. также не имеющие ценность
# Таких юнитов оказывается 375(9% от датасета), их удаление значительно очистит датасет

    if (isdotdigit(source)):
        dds += 1
    if (isdotdigit(target)):    
        ddt += 1

# Переменная original count для запоминания изначального количества юнитов в датасете

original_count = count

print("Пустых:", b)
print("Source-число:", s)
print("Target-число:", t)
print("Source меньше 6 символов:", trash)
print("В source только цифры и точки:", dds)
print("В target только цифры и точки:", ddt)
print("Количество юнитов:", count)

5 См.
5 See
________
ATC
ATS
________
B.
V.
________
№
No.
________
№ п/п
No.
________
А
A.
________
А.
A.
________
А.В.
A. V.
________
А.Ю.
A.Yu.
________
АБС
ABS
________
Акт
Certificate
________
АТС
ATS
________
Б.
B.
________
б/н
w/n
________
ББП
UPS
________
В.
V.
________
В.А.
V.A. 
________
Г.
G.
________
Г.И.
G.I.
________
ГО
HQ
________
гор.
.
________
Д.
D.
________
Дата
Date
________
ДБО
RB
________
Доп.
Extra
________
Е.
E.
________
ед
ba
________
И.
I.
________
ИБП
UPS
________
Исп.
Prepared by:
________
Исп.:
Prepared by:
________
К.Ю.
K.Yu.
________
Копия
copy
________
Л.
L.
________
Л.А.
L.A.
________
Л.И.
L.I.
________
Лица:
Persons: 
________
М.
M.
________
М.:
Moscow:
________
М.В.
M.V.
________
МПБ
International Industrial Bank
________
Н.
N.
________
Н/Д
N/a
________
нет
n/a
________
О.
O.
________
О.И.
O.I.
________
ОЗУ
RAM
________
П.
P.
________
П.А.
P.A.
________
п/п
No.
________
Р.
R.
________
С
S.
________
С.
S.
________
Свитч
Switch
________
свич
switch
____

#### Удаляем вычисленные невалидные юниты 

In [6]:
for node in tmx_file.getunits():
    source = node.getid()
    target = node.gettarget()
    
# Удаляем юниты с target-число   

    if target.isnumeric():
        tmx_file.removeunit(node)    

# Удаляем юниты с source меньше 6 символов

    elif (len(source) < 6):
        tmx_file.removeunit(node)
        
# Удаляем юниты только c цифрами и точками

    elif (isdotdigit(source)):
        tmx_file.removeunit(node)
    elif (isdotdigit(target)):
        tmx_file.removeunit(node)

#### Взяв список символов, валидных для русской и английской раскладки (включая space), я могу вычислить юниты, которые содержат порченые символы

In [7]:
count = 0
unknown = 0

_chars = u"~!@#$%^&qwertyuiop[]asdfghjkl;'zxcvbnm,./QWERTYUIOP{}ASDFGHJKL:\"|ZXCVBNM<>?ё!\"№;%:?йцукенгшщзхъфывапролджэячсмитьбю.ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭ/ЯЧСМИТЬБЮ, -1234567890()«—»–…+“”’€*©"

for node in tmx_file.unit_iter():
    flag = 1
    count += 1
    source = node.getid()
    target = node.gettarget()

    for s in source:
        if s not in _chars:
            print(s)
            print(source)
            print(target)
            print("-------------------------------------------------------------------")
            unknown += 1
            flag = 0
            break;
    for t in target:
        if t not in _chars:
            print(t)
            print(source)
            print(target)
            print("-------------------------------------------------------------------")
            if flag != 0:
                unknown += 1
            break;

print("Unknown:", unknown)
print("Количество юнитов:", count)

 
- возбужденное 05.11.15 ГСУ СК России уголовное дело № 201/404192-15 по ч. 3 ст. 33, ч. 2 ст. 201 (в отношении Пугачева С.В) и по ч. 2 ст. 201 (в отношении Злобина А.С.) УК РФ (см. п. 6).
- criminal case No. 201/404192-15 initiated by the Central Investigation Department of the Investigative Committee of the Russian Federation on May 05, 2015 under Part 3 Article 33, Part 2 Article 201 (in relation to S.V. Pugachev) and under Part 2 Article 201 (in relation to A.S. Zlobin) of the Criminal Code of the Russian Federation (see Clause 6).
-------------------------------------------------------------------
 
- возбужденное 27.11.13 ГСУ СК России уголовное дело № 201/813405-13 по ч. 4 ст. 159 УК РФ (см. п. 4);
- criminal case No. 201/813405-13 initiated by the Central Investigation Department of the Investigative Committee of the Russian Federation on November 27, 2013 under Part 4 Article 159 of the Criminal Code of the Russian Federation (see Clause 4);
----------------------------------

_
Комиссия по удалению электронных баз данных кредитных организаций, действующая на основании Приказа Генерального директора Агентства № ____ от _________ г., составила настоящий акт о том, что в связи с внесением ___________ г. в Единый государственный реестр юридических лиц записи о государственной регистрации (наименование кредитной организации) в связи с ее ликвидацией произведено удаление резервных копий электронной базы данных и базы данных бухгалтерского учета конкурсного производства (ликвидации) кредитной организации.
The Commission for Deletion of Electronic Databases of Credit Institutions acting on the basis of the Agency's General Director's Order No. ____ of _______________ has drawn up the present Certificate on deletion of the backup copies of the electronic database and the accounting database of the credit institution's liquidation procedure (liquidation) in connection with the entry made ___________ in the Unified State Register of Legal Entities in connection with i

 
Написала претензию в компанию!
I wrote a complaint to the company!   
-------------------------------------------------------------------
 
Светлана Светлана Сергеевна (02.11.2016 20:13)
Svetlana Svetlana Sergeyevna (02.11.2016 20:13)   
-------------------------------------------------------------------
 
Предоставил накладные, чеки и прочее.
I provided invoices, checks and more.   
-------------------------------------------------------------------
Unknown: 137
Количество юнитов: 3603


In [8]:
#Их оказалось всего 137(3,3% от датасета), также удаляем их

for node in tmx_file.getunits():
    flag = 1
    source = node.getid()
    target = node.gettarget()
    
    for s in source:
        if s not in _chars:
            tmx_file.removeunit(node)
            flag = 0
            break;
    for t in target:
        if t not in _chars:
            if flag != 0:
                tmx_file.removeunit(node)
            break;

#### С помощью регулярного выражения найдем юниты с ссылками

In [9]:
import re

c = 0

for node in tmx_file.getunits():
    source = node.getid()
    result = re.search(r'https?:?\/?\/?\w*.?\w+\.\w+\/[\d\w-]+\/?[\w\d-]*', source)
    if result != None:
        c += 1
        print(source)
        print(result)
        print("------------------------------------------------------------------------------------------")
print("Количество ссылок:", c)  

https://www.kommersant.ru/doc/1077783
<re.Match object; span=(0, 37), match='https://www.kommersant.ru/doc/1077783'>
------------------------------------------------------------------------------------------
Газета "Коммерсантъ" №213 (https://kommersant.ru/daily/41831) от 24.11.2008, стр. 7
<re.Match object; span=(27, 60), match='https://kommersant.ru/daily/41831'>
------------------------------------------------------------------------------------------
https://style.rbc.ru/impressions/571f2f2f9a79473d66b837c7
<re.Match object; span=(0, 57), match='https://style.rbc.ru/impressions/571f2f2f9a79473d>
------------------------------------------------------------------------------------------
Количество ссылок: 3


In [10]:
#Удаляем найденные юниты с ссылками

for node in tmx_file.getunits():
    source = node.getid()
    result = re.search(r'https?:?\/?\/?\w*.?\w+\.\w+\/[\d\w-]+\/?[\w\d-]*', source)
    if result != None:
        tmx_file.removeunit(node)

#### Проверим, насколько сократился наш датасет после очистки

In [11]:
count = 0

for node in tmx_file.getunits():
    count += 1

units = original_count - count
percent = 100 - (count * 100 // original_count)
    
print("Изначальное количество юнитов:", original_count)    
print("Количество юнитов после очистки:", count)
print('Датасет сократился на {} юнита или {}%'.format(units, percent)) 

Изначальное количество юнитов: 4045
Количество юнитов после очистки: 3463
Датасет сократился на 582 юнита или 15%


#### Сохраняем изменения нашего tmx файла

In [ ]:
tmx_file.save()